In [9]:
import cv2
import numpy as np

def image_stitching(image1, image2):
    # 转换图像为灰度图
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # 构建图像金字塔
    dog = cv2.createBackgroundSubtractorMOG2()
    pyramid1 = [gray1]
    pyramid2 = [gray2]
    for _ in range(4):
        gray1 = cv2.pyrDown(gray1)
        gray2 = cv2.pyrDown(gray2)
        pyramid1.append(gray1)
        pyramid2.append(gray2)

    # 初始化SIFT检测器
    sift = cv2.SIFT_create()

    # 寻找关键点和描述符
    keypoints1, descriptors1 = sift.detectAndCompute(pyramid1[-1], None)
    keypoints2, descriptors2 = sift.detectAndCompute(pyramid2[-1], None)

    # 匹配关键点
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)

    # 应用比率测试以消除错误匹配
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # 提取匹配的关键点的坐标
    points1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    points2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # 使用RANSAC算法估计图像间的单应性矩阵
    H, _ = cv2.findHomography(points1, points2, cv2.RANSAC)

    # 调整图像大小以匹配拼接后的图像
    height, width = pyramid1[0].shape
    stitched_image_size = (width, height)
    resized_image2 = cv2.resize(pyramid2[-1], stitched_image_size)

    # 将第二张图像变换到第一张图像的视角
    stitched_image = cv2.warpPerspective(resized_image2, H, stitched_image_size)
    

    # 将第一张图像与变换后的第二张图像叠加
    stitched_image[:height, :width] = resized_image2

    return stitched_image

# 读取两张图像
image1 = cv2.imread('images/1_1.JPG')
image2 = cv2.imread('images/1_2.JPG')

# 调用图像拼接函数
result_image = image_stitching(image1, image2)

# 显示结果
cv2.imshow('Stitched Image', result_image)
cv2.imwrite('results/DOG.jpg',result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


stitched_image_size: (3024, 4032)
resized_image2_size: (252, 189)
